In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('''C:\\Users\\user\\Downloads\\a_steam_data_2021_2025.csv''')

In [ ]:
data['genres'] = data['genres'].fillna(data.groupby('categories')['genres'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan))
data['genres'] = data['genres'].replace(['nan', 'None', ''], np.nan)
data['categories'] = data['categories'].replace(['nan', 'None', ''], np.nan)
data = data.dropna(subset=['genres', 'categories'])
data['developer'] = data['developer'].fillna('Noname')
data['publisher'] = data['publisher'].fillna('Noname')

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'], format='mixed', errors='coerce')
data['release_date'] = data['release_date'].fillna(pd.to_datetime('Aug 1 ' + data['release_year'].astype(str), format='%b %d %Y', errors='coerce'))       
data['genres'] = data['genres'].astype(str).str.split(';')
data['categories'] = data['categories'].astype(str).str.split(';')